# Segmenting and Clustering Neighborhoods in Toronto
**Applied Data Science Capstone**

## Webscraping Postal Codes

In the first part of this assignment, we will scrape Wikipedia for Postal Codes for various Canadian neighborhoods. 

Since this was previously described in an earlier assignment, I will just post the code here without explanation.

In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url_wiki = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url_wiki).text
soup = BeautifulSoup(page, 'lxml')
table = soup.find_all("table")[0]

header = table.find_all("tr")[0].find_all("th")
cols = [i.text for i in header]
cols[2] = cols[2].replace("\n", "")

table_rows = table.find_all("tr")
rows = []
for row in range(1,len(table_rows)):
    rows.append([i.text for i in table_rows[row].find_all("td")])
    
vals = [[] for i in cols]
for row in rows:
    if row[1] != "Not assigned":
        for elem in range(len(row)):
            vals[elem].append(row[elem])            
vals[2] = [i.replace("\n", "") for i in vals[2]]

df_post = pd.DataFrame({name:values for name, values in zip(cols, vals)})
df_post = df_post.groupby(["Postcode", "Borough"])["Neighbourhood"].apply(lambda x: ','.join(x)).reset_index()


## Adding Latitude & Longitude Coordinates

Again, I will just post the code here since this was described in more detail in the earlier submission.

In [14]:
coords = pd.read_csv("/home/brkalltheway/Dropbox/Coursera/IBM/capstone/Geospatial_Coordinates.csv")
coords.rename(columns={"Postal Code" : "Postcode"}, inplace=True)
df = pd.merge(df_post, coords, on="Postcode")


## Clustering Toronto

In this section, we will cluster the different boroughs in Toronto. First, let us find all of the boroughs that contain `"Toronto"` in their name.

In [21]:
tor_bor = [i for i in df["Borough"].unique() if "Toronto" in i]
tor_bor

['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']

Now, filter the DataFrame to only contain these areas.

In [26]:
df_tor = df[df.Borough.isin(tor_bor)].reset_index()
df_tor.head()

,index,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,37,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,41,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,42,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,43,M4M,East Toronto,Studio District,43.659526,-79.340923
4,44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


### Getting Data

Now, we can start obtain data on the various venues around each Neighborhood. Let us define our Foursquare Credentials and set some defaults for the API.

In [34]:
CLIENT_ID = "0RFHGZSEYYV5ENUITHZHT0SYU4LJ1OWV1DS0QNISBJUU3UED"
CLIENT_SECRET = "HOBDNO0K2MQTEA05DPYGCECCT4QMZ3OVGQHOBU4RDW42SXNY"
VERSION = "20191001"
LIMIT = 1000
RADIUS = 1000

With the credentials, we can define a function that queries the API, pulls out the relevant information, and formats it to a pandas DataFrame. This function takes the borough, neighborhood, latitude and longitude.

In [35]:
def getVenues(bor, neigh, lat, lng):
    url = "https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat, lng,
        RADIUS,
        LIMIT        
    )
    results = requests.get(url).json()["response"]["groups"][0]['items']
    venues  = []
    venues.append([(
            bor, neigh, lat,lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'], 
            v['venue']['categories'][0]['name']) for v in results])
    
    nvenues = pd.DataFrame([item for venues in venues for item in venues])
    nvenues.columns = ["Borough", "Neighborhood", "Neigh Lat", "Neigh Long", "Venue", "Lat", "Long", "Category"]
    return(nvenues)

Next, we initialize an empty DataFrame in which to store the results.

In [121]:
df_ven = pd.DataFrame(columns=["Borough", "Neighborhood", "Neigh Lat", "Neigh Long", "Venue", "Lat", "Long", "Category"])
df_ven

,Borough,Neighborhood,Neigh Lat,Neigh Long,Venue,Lat,Long,Category


Now, we loop across each row in `df_tor` and collect all of the venues within 1 km. 

In [122]:
for row in range(df_tor.shape[0]):
    vens = getVenues(df_tor["Borough"][row], df_tor["Neighbourhood"][row], df_tor["Latitude"][row], df_tor["Longitude"][row])
    df_ven = pd.concat([df_ven, vens])
df_ven.head()

,Borough,Neighborhood,Neigh Lat,Neigh Long,Venue,Lat,Long,Category
0,East Toronto,The Beaches,43.676357,-79.293031,Tori's Bakeshop,43.672114,-79.290331,Vegetarian / Vegan Restaurant
1,East Toronto,The Beaches,43.676357,-79.293031,The Beech Tree,43.680493,-79.288846,Gastropub
2,East Toronto,The Beaches,43.676357,-79.293031,The Fox Theatre,43.672801,-79.287272,Indie Movie Theater
3,East Toronto,The Beaches,43.676357,-79.293031,Ed's Real Scoop,43.672630,-79.287993,Ice Cream Shop
4,East Toronto,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail



### Wrangling Data

Note that there are more than 200 different categories to use for clustering only 38 Neighborhoods. Let us reduce this dimension by combining different categories. First, let us look at the different categories.

In [135]:
list(df_ven["Category"].unique())

['Vegetarian / Vegan Restaurant',
 'Gastropub',
 'Indie Movie Theater',
 'Ice Cream Shop',
 'Trail',
 'Bakery',
 'Bagel Shop',
 'Health Food Store',
 'Toy / Game Store',
 'French Restaurant',
 'Coffee Shop',
 'Breakfast Spot',
 'Park',
 'Beach',
 'Pub',
 'Mexican Restaurant',
 'Bar',
 'Cupcake Shop',
 'Tea Room',
 'Japanese Restaurant',
 'Liquor Store',
 'Diner',
 'Nail Salon',
 'Juice Bar',
 'Caribbean Restaurant',
 'Bookstore',
 'Burger Joint',
 'Grocery Store',
 'Chocolate Shop',
 'Jazz Club',
 'Thai Restaurant',
 'Sandwich Place',
 'Indian Restaurant',
 'Ramen Restaurant',
 'Pharmacy',
 'Greek Restaurant',
 'Café',
 'Asian Restaurant',
 'Pizza Place',
 'Beer Store',
 'Bank',
 'Electronics Store',
 'Shoe Store',
 'Flower Shop',
 'Restaurant',
 'Scenic Lookout',
 'Salon / Barbershop',
 'Sporting Goods Shop',
 'Tree',
 'Shopping Mall',
 'Cosmetics Shop',
 'Italian Restaurant',
 'Brewery',
 'Yoga Studio',
 'Fruit & Vegetable Store',
 'Spa',
 'Dessert Shop',
 'Concert Hall',
 'Furniture

After looking at the list, I decided to make a handful of groupings based on restaurants and bars to potentially highlight certain areas to cluster. I probably misplaced some items and there are likely items that I should have grouped as well that I did not. 

In [171]:
cat_replace = {
    "Café" : "Coffee Shop",
    "Ice Cream Shop" : "Dessert Shop",
    "Cupcake Shop" : "Dessert Shop",
    "Chocolate Shop" : "Dessert Shop",
    "Frozen Yogurt Shop" : "Dessert Shop",
    "Gastropub" : "Bar",
    "Pub" : "Bar",
    "Brewery" : "Bar",
    "Sports Bar" : "Bar",
    "Club" : "Bar",
    "Jazz Club" : "Bar",
    "Nightclub" : "Bar",
    "Cocktail Bar" : "Bar",
    "Speakeasy" : "Bar",
    "Whisky Bar" : "Bar",
    "Beach Bar" : "Bar",
    "Comedy Club" : "Bar",
    "Indie Movie Theater" : "Theater",
    "Indie Theater" : "Theater",
    "Movie Theater" : "Theater",
    "Bagel Shop" : "Bakery",
    "Pastry Shop" : "Bakery",
    "Burger Joint" : "American Restaurant",
    "Sandwich Place" : "American Restaurant",
    "Pizza Place" : "American Restaurant",
    "BBQ Joint" : "American Restaurant",
    "Hot Dog Joint" : "American Restaurant",
    "Steakhouse" : "American Restaurant",
    "New American Restaurant" : "American Restaurant",
    "Southern / Soul Food Restaurant" : "American Restaurant",
    "Cajun / Creole Restaurant" : "American Restaurant",
    "Mac & Cheese Joint" : "American Restaurant",
    "French Restaurant" : "European Restaurant",
    "Greek Restaurant" : "European Restaurant",
    "Italian Restaurant" : "European Restaurant",
    "German Restaurant" : "European Restaurant",
    "Polish Restaurant" : "European Restaurant",
    "Spanish Restaurant" : "European Restaurant",
    "Belgian Restaurant" : "European Restaurant",
    "Eastern European Restaurant" : "European Restaurant",
    "Modern European Restaurant" : "European Restaurant",
    "Tapas Restaurant" : "European Restaurant",
    "Portuguese Restaurant" : "European Restaurant",
    "Mediterranean Restaurant" : "European Restaurant",
    "Turkish Restaurant" : "Middle Eastern Restaurant",
    "Falafel Restaurant" : "Middle Eastern Restaurant",
    "Pakastani Restaurant" : "Middle Eastern Restaurant",
    "Persian Restaurant" : "Middle Eastern Restaurant",
    "Afghan Restaurant" : "Middle Eastern Restaurant",
    "Jewish Restaurant" : "Middle Eastern Restaurant",
    "Japanese Restaurant" : "Asian Restaurant",
    "Ramen Restaurant" : "Asian Restaurant",
    "Sushi Restaurant" : "Asian Restaurant",
    "Thai Restaurant" : "Asian Restaurant",
    "Indian Restaurant" : "Asian Restaurant",
    "Indian Chinese Restaurant" : "Asian Restaurant",
    "Chinese Restaurant" : "Asian Restaurant",
    "Vietnamese Restaurant" : "Asian Restaurant",
    "Korean Restaurant" : "Asian Restaurant",
    "Indonesian Restaurant" : "Asian Restaurant",
    "Cantonese Restaurant" : "Asian Restaurant",
    "Filipino Restaurant" : "Asian Restaurant",
    "Taiwanese Restaurant" : "Asian Restaurant",
    "Dumpling Restaurant" : "Asian Restaurant",
    "Noodle House" : "Asian Restaurant",
    "Udon Restaurant" : "Asian Restaurant",
    "Malay Restaurant" : "Asian Restaurant",
    "Hawaiian Restaurant" : "Asian Restaurant",
    "Tibetan Restaurant" : "Asian Restaurant",
    "Cuban Restaurant" : "Latin Restaurant",
    "Mexican Restaurant" : "Latin Restaurant",
    "Burrito Restaurant" : "Latin Restaurant",
    "Taco Restaurant" : "Latin Restaurant",
    "Brazilian Restaurant" : "Latin Restaurant",
    "South American Restaurant" : "Latin Restaurant",
    "Fruit & Vegetable Store" : "Grocery Store"
}

Now that I have some groupings. I will go through and replace the items and show the new count.

In [172]:
df_ven2 = df_ven

for key, value in cat_replace.items():
    df_ven2.replace(to_replace=key, value=value, inplace=True)

df_ven2.groupby("Category")["Category"].count().sort_values(ascending=False)

Category
Coffee Shop            421
Asian Restaurant       272
Bar                    229
American Restaurant    210
European Restaurant    185
                      ... 
Hotel Bar                1
Intersection             1
Light Rail Station       1
Martial Arts Dojo        1
Accessories Store        1
Name: Category, Length: 202, dtype: int64

In [173]:
cats = pd.get_dummies(df_ven2["Category"])
cats["Neighborhood"] = df_ven2["Neighborhood"]

n = list(cats.columns).index("Neighborhood")
cols_fixed = [cats.columns[n]] + list(cats.columns[:n]) + list(cats.columns[n+1:])
cats = cats[cols_fixed]
cats.head()

,Neighborhood,Accessories Store,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,...,Tree,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


To obtain the relative number of particular venues in each Neighborhood, we take the grouped mean. We can also print the top 5 types of venues in each Neighborhood for comparison.

In [174]:
cats_grouped = cats.groupby("Neighborhood").mean().reset_index()
n_top_venues = 5

for hood in cats_grouped["Neighborhood"]:
    print("----"+hood+"----")
    tmp = cats_grouped[cats_grouped["Neighborhood"] == hood].T.reset_index()
    tmp.columns = ["venue", 'freq']
    tmp = tmp.iloc[1:]
    tmp['freq'] = tmp['freq'].astype(float)
    tmp = tmp.round({'freq':2})
    print(tmp.sort_values('freq', ascending=False).reset_index(drop=True).head(n_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                 venue  freq
0          Coffee Shop  0.14
1     Asian Restaurant  0.10
2  American Restaurant  0.09
3                  Bar  0.07
4              Theater  0.06


----Berczy Park----
                 venue  freq
0          Coffee Shop  0.15
1                Hotel  0.06
2  European Restaurant  0.06
3                  Bar  0.06
4     Asian Restaurant  0.05


----Brockton,Exhibition Place,Parkdale Village----
                    venue  freq
0             Coffee Shop  0.14
1        Asian Restaurant  0.07
2                     Bar  0.07
3     American Restaurant  0.05
4  Furniture / Home Store  0.04


----Business Reply Mail Processing Centre 969 Eastern----
                 venue  freq
0  American Restaurant  0.13
1                  Bar  0.11
2                 Park  0.09
3     Asian Restaurant  0.06
4  European Restaurant  0.06


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
         

Next, we find out what the most common types of venues are in each Neighborhood and place these in a DataFrame. These will be listed as `"Rank"`. We will push the clustered results back to this DataFrame later.

In [184]:
import numpy as np

def return_most_common_venues(row, n):
    row_cat = row.iloc[1:].sort_values(ascending=False)
    return(row_cat.index.values[0:n])

n_venues = 10
columns = ["Neighborhood"]
columns.extend(["Rank"+str((i+1)) for i in range(n_venues)])

neigh_venues = pd.DataFrame(columns=columns)
neigh_venues["Neighborhood"] = cats_grouped['Neighborhood']

for ind in np.arange(cats_grouped.shape[0]):
    neigh_venues.iloc[ind, 1:] = return_most_common_venues(cats_grouped.iloc[ind, :], n_venues)
    
neigh_venues.head()
    

,Neighborhood,Rank1,Rank2,Rank3,Rank4,Rank5,Rank6,Rank7,Rank8,Rank9,Rank10
0,"Adelaide,King,Richmond",Coffee Shop,Asian Restaurant,American Restaurant,Bar,Theater,Hotel,European Restaurant,Beer Bar,Restaurant,Clothing Store
1,Berczy Park,Coffee Shop,European Restaurant,Hotel,Bar,Asian Restaurant,Beer Bar,American Restaurant,Bakery,Restaurant,Park
2,"Brockton,Exhibition Place,Parkdale Village",Coffee Shop,Asian Restaurant,Bar,American Restaurant,Furniture / Home Store,European Restaurant,Bakery,Restaurant,Thrift / Vintage Store,Gift Shop
3,Business Reply Mail Processing Centre 969 Eastern,American Restaurant,Bar,Park,European Restaurant,Coffee Shop,Asian Restaurant,Pet Store,Burrito Place,Fish & Chips Shop,Bistro
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Coffee Shop,Harbor / Marina,Dance Studio,Scenic Lookout,Sculpture Garden,Track,Park,Garden,Dog Run,Airport Lounge



### Clustering

Now, we will cluster each of the Neighborhoods into one of 8 different clusters using KMeans. 8 was chosen as smaller number of groups were unable to detect certain key differences between neighborhoods.

In [185]:
from sklearn.cluster import KMeans

nclusters = 8
df_cluster = cats_grouped.drop("Neighborhood", 1)

kmeans = KMeans(n_clusters=nclusters, random_state=0).fit(df_cluster)
kmeans.labels_

array([3, 3, 1, 2, 5, 2, 2, 1, 2, 2, 3, 2, 0, 2, 3, 0, 3, 2, 1, 3, 0, 2,
       4, 1, 2, 0, 2, 7, 6, 0, 2, 3, 3, 2, 1, 1, 2, 0], dtype=int32)

Once we have the cluster labels, we insert those labels back into our DataFrame that contains a variety of different variables for each Neighborhood.

In [186]:
neigh_venues2 = neigh_venues
neigh_venues2.insert(0, "Cluster", kmeans.labels_)

toronto = df_tor.rename(columns={"Neighbourhood" : "Neighborhood"})
toronto = toronto.join(neigh_venues2.set_index("Neighborhood"), on="Neighborhood")
toronto.head()

,index,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster,Rank1,Rank2,Rank3,Rank4,Rank5,Rank6,Rank7,Rank8,Rank9,Rank10
0,37,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Bar,Asian Restaurant,Coffee Shop,American Restaurant,Bakery,Beach,Breakfast Spot,Dessert Shop,European Restaurant,Caribbean Restaurant
1,41,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,0,European Restaurant,Coffee Shop,American Restaurant,Bar,Asian Restaurant,Dessert Shop,Bank,Diner,Yoga Studio,Grocery Store
2,42,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,2,Asian Restaurant,Coffee Shop,American Restaurant,Bar,Beach,Theater,Harbor / Marina,Grocery Store,Fast Food Restaurant,Burrito Place
3,43,M4M,East Toronto,Studio District,43.659526,-79.340923,2,Coffee Shop,Bar,Asian Restaurant,American Restaurant,European Restaurant,Bakery,Diner,Arts & Crafts Store,Middle Eastern Restaurant,Park
4,44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Coffee Shop,Gym / Fitness Center,Trail,Park,College Gym,Bookstore,College Quad,Antique Shop,Diner,Ethiopian Restaurant


Now, let us see where these clusters are located relative to each other in Toronto. To do so, we will get the latitude and longitude coordinates of Toronto using `geopy`.

In [180]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="cantor").geocode("Toronto, Canada")
latitude = geolocator.latitude
longitude = geolocator.longitude
print("Lat: " + str(latitude) + "\nLng: " + str(longitude))

Lat: 43.653963
Lng: -79.387207


Now that we have the coordinates, we can use `folium` to plot the location and insert dots corresponding to each Neighborhood. 

In [187]:
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(nclusters)
ys = [i + x + (i*x)**2 for i in range(nclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto['Latitude'], toronto['Longitude'], toronto['Neighborhood'], toronto['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

We can see that we have some very distinct groups. The downtown coastal areas are grouped together. As we move further from that center, the areas become more similar to each other with a handful of unique areas, such as the airport. 

We can examine each in more detail as seen in the code below.

In [189]:
toronto.loc[toronto["Cluster"] == 0, toronto.columns[[2] + list(range(7, toronto.shape[1]))]]

,Borough,Rank1,Rank2,Rank3,Rank4,Rank5,Rank6,Rank7,Rank8,Rank9,Rank10
1,East Toronto,European Restaurant,Coffee Shop,American Restaurant,Bar,Asian Restaurant,Dessert Shop,Bank,Diner,Yoga Studio,Grocery Store
5,Central Toronto,Coffee Shop,Asian Restaurant,European Restaurant,Dessert Shop,Fast Food Restaurant,Bar,American Restaurant,Gym,Pharmacy,Sporting Goods Shop
6,Central Toronto,Coffee Shop,European Restaurant,Asian Restaurant,Sporting Goods Shop,Park,Diner,Latin Restaurant,Flower Shop,Tea Room,Salon / Barbershop
13,Downtown Toronto,Coffee Shop,European Restaurant,Bar,Asian Restaurant,Park,Theater,Breakfast Spot,Dessert Shop,Diner,Bakery
31,West Toronto,Coffee Shop,Park,Bar,European Restaurant,Asian Restaurant,Gourmet Shop,Latin Restaurant,Supermarket,Grocery Store,Bakery
36,West Toronto,Coffee Shop,American Restaurant,Bar,European Restaurant,Bakery,Asian Restaurant,Park,Middle Eastern Restaurant,Dessert Shop,Smoothie Shop


In [190]:
toronto.loc[toronto["Cluster"] == 1, toronto.columns[[2] + list(range(7, toronto.shape[1]))]]

,Borough,Rank1,Rank2,Rank3,Rank4,Rank5,Rank6,Rank7,Rank8,Rank9,Rank10
0,East Toronto,Bar,Asian Restaurant,Coffee Shop,American Restaurant,Bakery,Beach,Breakfast Spot,Dessert Shop,European Restaurant,Caribbean Restaurant
24,Central Toronto,European Restaurant,American Restaurant,Coffee Shop,Asian Restaurant,Bar,Vegetarian / Vegan Restaurant,Bakery,Middle Eastern Restaurant,Restaurant,Gym
25,Downtown Toronto,Coffee Shop,Asian Restaurant,Bar,Bakery,American Restaurant,Vegetarian / Vegan Restaurant,European Restaurant,Restaurant,Dessert Shop,Bookstore
26,Downtown Toronto,Bar,Coffee Shop,Asian Restaurant,Vegetarian / Vegan Restaurant,Bakery,Dessert Shop,American Restaurant,European Restaurant,Latin Restaurant,Gaming Cafe
32,West Toronto,Bar,Coffee Shop,Asian Restaurant,American Restaurant,European Restaurant,Bakery,Restaurant,Men's Store,Theater,Bookstore
33,West Toronto,Coffee Shop,Asian Restaurant,Bar,American Restaurant,Furniture / Home Store,European Restaurant,Bakery,Restaurant,Thrift / Vintage Store,Gift Shop


In [191]:
toronto.loc[toronto["Cluster"] == 2, toronto.columns[[2] + list(range(7, toronto.shape[1]))]]

,Borough,Rank1,Rank2,Rank3,Rank4,Rank5,Rank6,Rank7,Rank8,Rank9,Rank10
2,East Toronto,Asian Restaurant,Coffee Shop,American Restaurant,Bar,Beach,Theater,Harbor / Marina,Grocery Store,Fast Food Restaurant,Burrito Place
3,East Toronto,Coffee Shop,Bar,Asian Restaurant,American Restaurant,European Restaurant,Bakery,Diner,Arts & Crafts Store,Middle Eastern Restaurant,Park
7,Central Toronto,Asian Restaurant,Coffee Shop,European Restaurant,Bar,American Restaurant,Dessert Shop,Gym,Fast Food Restaurant,Restaurant,Bookstore
8,Central Toronto,European Restaurant,Coffee Shop,Asian Restaurant,American Restaurant,Park,Gym,Grocery Store,Bakery,Bar,Playground
9,Central Toronto,Coffee Shop,Asian Restaurant,European Restaurant,American Restaurant,Park,Bar,Bakery,Gym / Fitness Center,Spa,Gym
11,Downtown Toronto,Asian Restaurant,Coffee Shop,Bar,Park,American Restaurant,Diner,Restaurant,Pet Store,Butcher,Caribbean Restaurant
12,Downtown Toronto,Asian Restaurant,Coffee Shop,American Restaurant,European Restaurant,Bar,Middle Eastern Restaurant,Park,Men's Store,Bubble Tea Shop,Bookstore
14,Downtown Toronto,Coffee Shop,Asian Restaurant,American Restaurant,Clothing Store,Bar,European Restaurant,Middle Eastern Restaurant,Diner,Theater,Cosmetics Shop
17,Downtown Toronto,Coffee Shop,Asian Restaurant,American Restaurant,European Restaurant,Bar,Theater,Bubble Tea Shop,Park,Dessert Shop,Burrito Place
23,Central Toronto,Coffee Shop,Park,American Restaurant,Asian Restaurant,European Restaurant,Middle Eastern Restaurant,Trail,Bakery,Bank,Pharmacy


In [192]:
toronto.loc[toronto["Cluster"] == 3, toronto.columns[[2] + list(range(7, toronto.shape[1]))]]

,Borough,Rank1,Rank2,Rank3,Rank4,Rank5,Rank6,Rank7,Rank8,Rank9,Rank10
15,Downtown Toronto,Coffee Shop,Bar,American Restaurant,Restaurant,European Restaurant,Asian Restaurant,Hotel,Bakery,Seafood Restaurant,Theater
16,Downtown Toronto,Coffee Shop,European Restaurant,Hotel,Bar,Asian Restaurant,Beer Bar,American Restaurant,Bakery,Restaurant,Park
18,Downtown Toronto,Coffee Shop,Asian Restaurant,American Restaurant,Bar,Theater,Hotel,European Restaurant,Beer Bar,Restaurant,Clothing Store
19,Downtown Toronto,Coffee Shop,Hotel,Bar,European Restaurant,American Restaurant,Asian Restaurant,Aquarium,Theater,Scenic Lookout,Park
20,Downtown Toronto,Coffee Shop,American Restaurant,Asian Restaurant,Hotel,European Restaurant,Bar,Concert Hall,Theater,Restaurant,Park
21,Downtown Toronto,Coffee Shop,Bar,Asian Restaurant,Hotel,American Restaurant,European Restaurant,Restaurant,Concert Hall,Salad Place,Deli / Bodega
28,Downtown Toronto,Coffee Shop,American Restaurant,Bar,Restaurant,Asian Restaurant,Beer Bar,Hotel,Bakery,European Restaurant,Seafood Restaurant
29,Downtown Toronto,Coffee Shop,American Restaurant,Asian Restaurant,Hotel,European Restaurant,Bar,Theater,Plaza,Beer Bar,Salad Place


In [193]:
toronto.loc[toronto["Cluster"] == 4, toronto.columns[[2] + list(range(7, toronto.shape[1]))]]

,Borough,Rank1,Rank2,Rank3,Rank4,Rank5,Rank6,Rank7,Rank8,Rank9,Rank10
4,Central Toronto,Coffee Shop,Gym / Fitness Center,Trail,Park,College Gym,Bookstore,College Quad,Antique Shop,Diner,Ethiopian Restaurant


In [194]:
toronto.loc[toronto["Cluster"] == 5, toronto.columns[[2] + list(range(7, toronto.shape[1]))]]

,Borough,Rank1,Rank2,Rank3,Rank4,Rank5,Rank6,Rank7,Rank8,Rank9,Rank10
27,Downtown Toronto,Coffee Shop,Harbor / Marina,Dance Studio,Scenic Lookout,Sculpture Garden,Track,Park,Garden,Dog Run,Airport Lounge


In [195]:
toronto.loc[toronto["Cluster"] == 6, toronto.columns[[2] + list(range(7, toronto.shape[1]))]]

,Borough,Rank1,Rank2,Rank3,Rank4,Rank5,Rank6,Rank7,Rank8,Rank9,Rank10
22,Central Toronto,Coffee Shop,Asian Restaurant,Bank,European Restaurant,Bakery,Pharmacy,Bar,Skating Rink,Pet Store,Gym Pool


In [196]:
toronto.loc[toronto["Cluster"] == 7, toronto.columns[[2] + list(range(7, toronto.shape[1]))]]

,Borough,Rank1,Rank2,Rank3,Rank4,Rank5,Rank6,Rank7,Rank8,Rank9,Rank10
10,Downtown Toronto,Coffee Shop,American Restaurant,Bank,Park,Asian Restaurant,Grocery Store,Pie Shop,Candy Store,Breakfast Spot,Office
